Install necessary packages:

In [ ]:
!pip install selenium

Import the following modules

In [122]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

Open the given URL

In [52]:
#options = webdriver.ChromeOptions()
#options.add_argument('--headless=new')
driver = webdriver.Chrome()
url = 'https://www.accuweather.com/en/ph/baguio-city/1-262309_1_al/daily-weather-forecast/1-262309_1_al'

driver.get(url)
time.sleep(5)

Data Scraping

In [124]:
date_raw = driver.find_elements(By.CLASS_NAME, 'date') #extract the dates
temp_raw = driver.find_elements(By.CLASS_NAME, 'temp') #extract the temperature
precip_raw = driver.find_elements(By.CLASS_NAME, 'precip') #extract the precipitation probability 
condition_raw = driver.find_elements(By.CLASS_NAME, 'phrase') #extract the weather condition
realFeel_raw = driver.find_elements(By.CLASS_NAME, 'left') #extract the real feel temperature
uvIndAndWind_raw = driver.find_elements(By.CLASS_NAME, 'right') #extract the max UV index and wind

In [125]:
data = []
for i in range(len(temp)):
    [day, dateMMDD] = date_raw[3*i].text.split('\n')
    [tempHigh, tempLow] = temp_raw[i].text.split(' /')
    [_, realFeelTemp, _, realFeelShade] = realFeel_raw[i+1].text.split('\n')
    [_, maxUV, _, wind] = uvIndAndWind_raw[i+1].text.split('\n')
    [windDirection, windSpeed] = wind.split(' ', 1)
    data.append({
        'day': day,
        'date': dateMMDD,
        'tempHigh': tempHigh,
        'tempLow': tempLow,
        'condition': condition[i].text,
        'precip': precip[i].text,
        'realFeel': realFeelTemp,
        'realFeelShade': realFeelShade,
        'maxUV': maxUV,
        'windDirection': windDirection,
        'windSpeed': windSpeed
    })

df = pd.DataFrame(data)


In [129]:
#save the data to a csv file
df.to_csv('weather_data.csv', index=False)

Data Cleaning

In [150]:
weather_data = pd.read_csv('weather_data.csv')

#convert the date column to date format
weather_data['date'] = pd.to_datetime(weather_data['date'] + '/2024').dt.strftime('%m/%d/%Y')

weather_data['tempHigh'] = weather_data['tempHigh'].str.replace('°', '').astype(int)
weather_data['tempLow'] = weather_data['tempLow'].str.replace('°', '').astype(int)
weather_data['precip'] = weather_data['precip'].str.replace('%', '').astype(int)
weather_data['realFeel'] = weather_data['realFeel'].str.replace('°', '').astype(int)
weather_data['realFeelShade'] = weather_data['realFeelShade'].str.replace('°', '').astype(int)
weather_data['maxUV'] = weather_data['maxUV'].str.split(' ').str[0].astype(int)
weather_data['windSpeed'] = weather_data['windSpeed'].str.split(' ').str[0].astype(int)

#drop the day column
weather_data = weather_data.drop(columns=['day', 'condition'])

weather_data.head()

,date,tempHigh,tempLow,precip,realFeel,realFeelShade,maxUV,windDirection,windSpeed
0,07/25/2024,21,19,100,24,21,4,SW,11
1,07/26/2024,23,20,96,27,24,5,SW,9
2,07/27/2024,24,19,97,27,24,4,WSW,7
3,07/28/2024,22,18,95,25,22,4,SW,6
4,07/29/2024,22,19,96,26,23,5,WSW,7


In [151]:
#save the cleaned data to a csv file
weather_data.to_csv('weather_data_clean.csv', index=False)